This notebook aims to assess the generalization accuracy of a generated suffix, assuming a data-split was used during training.

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoTokenizer
import pandas as pd
import seaborn as sns
from types import SimpleNamespace
from datasets import Dataset
from os.path import join as oj
import pickle as pkl
import os
import dvu
dvu.set_style()
import analyze_utils
import sys
sys.path.append('..')
import data
from model_utils import prompt_classification

# Load results and plot

In [2]:
prompts_df = pkl.load('../results/autoprompt_sentiment/prompts.pkl')

TypeError: file must have 'read' and 'readline' attributes

In [ ]:
class fake_args:
    template_num_task_phrasing = 0
    max_dset_size = 1000
    max_digit = 10
    seed = 1
    train_split_frac = 0.75

    # these will be varied
    n_shots = 1
    task_name = 'add_two'


args = fake_args()
np.random.seed(args.seed)

ds = []
results_dir = '../results/generalization_acc/'
# checkpoints = []
checkpoints = ['gpt2-medium', 'EleutherAI/gpt-j-6B', 'gpt2-xl']
for n_shots in [1, 5, 10]:
    for checkpoint in checkpoints:  # , 'gpt2-xl']:['gpt2-medium']:
        save_name = f'baseline_accs_{checkpoint.replace("/", "___")}___nshots={n_shots}.pkl'
        d = pd.DataFrame.from_dict(pkl.load(open(
            oj(results_dir, save_name, 'rb'))))
        ds.append(deepcopy(d))
df = pd.concat(ds)
df['prompt'][df['prompt'] == ''] = 'no prompt'
df = df.sort_values(by=['task_name', 'prompt'])

for checkpoint in checkpoints:
    d = df[df.checkpoint == checkpoint]
    d = d[d.n_shots == 5]
    plt.figure(figsize=(6, 8))
    ax = sns.barplot(y='task_name', x='acc', data=d,
                     hue=(d['prompt'] + ', n_ex=' + d['n_shots'].astype(str)))
    ax.grid()
    plt.xlim(0, 100)
    plt.title(checkpoint)
    plt.savefig(
        oj(results_dir, f'baseline_accs_{checkpoint.replace("/", "___")}.pdf'), bbox_inches='tight')
    plt.show()


# Debug experiments

In [72]:
class fake_args:
    template_num_task_phrasing = 0
    max_dset_size = 1000
    max_digit = 10
    seed = 1
    train_split_frac = 0.75

    # these will be varied
    n_shots = 1
    task_name = 'add_two'
args = fake_args()
np.random.seed(args.seed)


task_names = ['task1191_food_veg_nonveg']
batch_sizes = {
    'gpt2-medium': 32,
    'EleutherAI/gpt-j-6B': 8,
    'EleutherAI/gpt-neox-20b': 1,
}
checkpoint = 'EleutherAI/gpt-j-6B'
d = defaultdict(list)
print('loading', checkpoint)
model = prompt_classification.create_model(checkpoint)

loading EleutherAI/gpt-j-6B


In [84]:
print('calculating accs...')
for task_name in tqdm(task_names):
    for prompt in ['manual']:
        for n_shots in [1]: #, 5]: 
                args.task_name = task_name
                args.n_shots = n_shots
                (dset, dset_test), check_answer_func, descr = data.get_data(
                    args, args.task_name, n_shots=args.n_shots, train_split_frac=args.train_split_frac)
                d['checkpoint'].append(checkpoint)
                d['prompt'].append(prompt)
                d['task_name'].append(task_name)
                d['n_shots'].append(n_shots)
                if prompt == 'manual':
                    prompt_actual = descr
                else:
                    prompt_actual = prompt
                d['prompt_actual'].append(prompt_actual)
                batch_size = batch_sizes.get(checkpoint, 16)
                if task_name == 'task107_splash_question_to_sql':
                    batch_size = max(1, batch_size//4)
                loss, acc = prompt_classification.test_model_on_task_with_prefix(
                    dset=dset, model=model, prefix=prompt_actual, multi_token=True, verbose=False,
                    batch_size=batch_size,
                )
                d['acc'].append(acc)

calculating accs...


100%|██████████| 1/1 [00:02<00:00,  2.72s/it]


# Manually inspect prompts

In [ ]:
for task_name in task_names:
    (dset, dset_test), check_answer_func, descr = data.get_data(
        args, task_name, n_shots=args.n_shots, train_split_frac=args.train_split_frac)
    print(task_name, descr, dset[0], end='\n\n')